In [1]:
import pickle
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
import nltk
import pandas as pd
import spacy
import time

# NOTE: stopwords include pronouns! TODO: make custom stop words list?
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/srutikanthan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize


stop_words = stopwords.words("english")

In [3]:
lmtzr = WordNetLemmatizer()

In [4]:

import string
import unicodedata
import re


def comments_clean(x):
    # define punctuation
    punctuations = '''"--"!()[]{};:'="\,<>./?@#$%^&*_~%'''

    # remove all punctuation from comments
    x['no_punct_col'] = x['comment'].copy() #The values in no_punct_col will be overwritten at the end of the for row loop
    i = 0
    for row in x['comment']: # looking at each row
        no_punct_str = ""
        for char in row:
            if char not in punctuations:
                no_punct_str = no_punct_str + char
        x['no_punct_col'][i] = no_punct_str # overwrite value in no_punct_col
        i += 1
    
    # lowercase, strip trailing whitespace, remove numbers, normalize text
    x['cleaned_text'] = x['no_punct_col'].str.lower() 
    x['cleaned_text'] = x['cleaned_text'].str.strip()
    x['cleaned_text'] = x['cleaned_text'].str.strip('\n')
    x['cleaned_text'] = x['cleaned_text'].str.strip('\n\n') # get rid of double returns/enters
    x['cleaned_text'] = x['cleaned_text'].str.replace(r'--', ' ')
    x['cleaned_text'] = x['cleaned_text'].str.replace(r'\.http.*', '')  
    x['no_punct_text'] = x['cleaned_text'].str.replace('\d+', '') # remove numbers
    x['no_punct_text'] = x['no_punct_text'].str.strip() # remove leading and trailing whitespace/escape chars
    x['no_punct_text'] = x['no_punct_text'].apply(lambda text: unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('ascii'))
    
    cleaned_comments = x['no_punct_text'].to_list()
    # lemmatized comments
    lemmatized = [[lmtzr.lemmatize(word) for word in word_tokenize(s)]
              for s in cleaned_comments]
    # stopwords and hyperlinks removed
    no_stopwords = [[word for word in l if word not in stop_words]
              for l in lemmatized]
    no_stopwords = [[word for word in l if 'http' not in word]
              for l in no_stopwords]
    
    return no_stopwords
        
    
        
            

In [5]:
comments_corpus = pickle.load(open("comments_corpus_RC_2011-08.pickle","rb"))

# change corpus to dataframe
subreddit_lst = []
comment_lst = []
time_lst = []
for key in comments_corpus:
    for comment, time in comments_corpus[key]:
        subreddit_lst.append(key)
        comment_lst.append(comment)
        time_lst.append(time)
        
comments_df = pd.DataFrame(list(zip(subreddit_lst, comment_lst, time_lst)),
                           columns = ['subreddit', 'comment', 'time created'])

In [6]:
lemm = comments_clean(comments_df)
lemm

[['post', 'picture', 'inside', 'book', 'lined'],
 ['inside', 'daenerys', 'whole', 'time'],
 ['wall', 'wasnt', 'built', 'keep', 'wildlings'],
 ['yeah',
  'probably',
  'episode',
  'need',
  'massively',
  'modified',
  'regard',
  'see',
  'good',
  'thing',
  'grrm',
  'one',
  'calling',
  'shot'],
 ['hah',
  'yeah',
  'guy',
  'saw',
  'researching',
  'making',
  'costume',
  'guy',
  'going',
  'amazing',
  'costume',
  'would',
  'like',
  'steal'],
 ['one', 'way', 'better', 'cut', 'narrated', 'though'],
 ['acokb',
  'theons',
  'uncle',
  'one',
  'meet',
  'theon',
  'return',
  'pyke',
  'take',
  'back',
  'castle',
  'meet',
  'father',
  'usually',
  'called',
  'damphair',
  'priest'],
 ['get',
  'people',
  'posting',
  'book',
  'people',
  'posting',
  'people',
  'posting',
  'book',
  'stop'],
 ['hint', 'stratego', 'look', 'rather', 'fun'],
 ['hope', 'budgeted', 'spray', 'tan'],
 ['mother',
  'child',
  'milk',
  'action',
  'might',
  'work',
  'dont',
  'see',
  'dr